In [28]:
import json, os
from pathlib import Path
from matplotlib import pyplot as plt

In [50]:
def season_adder(current: str) -> str:
    end_year = current.split('_')[1]
    return f'{end_year}_{str(int(end_year) + 1)}'

def version_adder(season: str, version: str) -> str:
    versions = ['start', 'summer_end', 'winter_end', 'end']
    next_version_num = versions.index(version) + 1
    if next_version_num > 3:
        next_version_num = 0
        season = season_adder(season)
    next_version = versions[next_version_num]
    return season, next_version

def best_version(season: str) -> str:
    

def players_path_generator(season: str, version: str) -> str:
    folder = Path(f'C:/Users/kazik/OneDrive/Desktop/Stuff/fifa/Data/{season}/')
    return folder / f'players/{version}.json'

def matches_path_generator(season: str, comp: str) -> str:
    folder = Path(f'C:/Users/kazik/OneDrive/Desktop/Stuff/fifa/Data/{season}/')
    return folder / f'matches/{comp}.json'

def loader(version: str, season: str) -> tuple:
    players_path = players_path_generator(season, version)
    players = json.load(open(players_path, encoding='utf-8'))
    matches = {}
    for comp in ['LaLiga', 'UCL', 'Copa']:
        file_path = matches_path_generator(season, comp)
        try:
            match = json.load(open(file_path, encoding='utf-8'))
            matches[comp] = match
        except json.JSONDecodeError:
            continue
    return players, matches

In [53]:
version = 'summer_end'
season = '25_26'

players, matches = loader(version, season)

In [32]:
def remove_sold(players: list) -> list:
    for player in players:
        if player['Status'] == 'Sold':
            players.remove(player)
    
    return players

def remove_loan(players: list) -> list:
    for player in players:
        if player['Status'] == 'On Loan':
            players.remove(player)
    return players